In [ ]:
import torch as tr
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import os

from vre import VRE, FFmpegVideo
from vre.representations import build_representations_from_cfg
from vre.utils import get_project_root, image_read, collage_fn, image_write, vre_yaml_load
from vre_repository import get_vre_repository

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
video = FFmpegVideo(get_project_root() / "resources/test_video.mp4")
video.file = get_project_root() / "resources/test_video.mp4"
print(video)

In [ ]:
os.environ["VRE_DEVICE"] = device = "cuda" if tr.cuda.is_available() else "cpu"
all_representations_dict = vre_yaml_load(Path.cwd() / "cfg.yaml")
representations = build_representations_from_cfg(all_representations_dict, get_vre_repository())
print(representations)

## Using VRE batched (default)
This goes representation by representation, which is more suited for entire videos to be parallelized, not frame by frame.

In [ ]:
vre = VRE(video, representations)
output_dir = Path.cwd() / "here"
shutil.rmtree(output_dir, ignore_errors=True)
print(vre)
start_frame = np.random.randint(0, len(video) - 5)
frames = range(start_frame, start_frame+1)


In [ ]:
# res = vre.run(output_dir=output_dir, frames=range(start_frame, start_frame+1), output_dir_exists_mode="skip_computed")
# frames = [x.name for x in Path(output_dir / "rgb/png").iterdir()]
# frame_name = np.random.permutation(frames)[0]
# imgs = [image_read(output_dir / r.name / "png" / frame_name) for r in vre.representations]
# collage = collage_fn(imgs, titles=[r.name for r in vre.representations], size_px=70, rows_cols=None)
# image_write(collage, f"collage_{frame_name}")
# plt.figure(figsize=(20, 10))
# plt.imshow(collage)
# plt.show()

## Using VRE streaming handcrafted
Soon this will be a real mode in VRE which goes frame by frame not representation by representation.

In [ ]:

display(vre.to_graphviz())
print(vre)
vre_streaming = vre.to_streaming()
res = vre_streaming[frames]
for i in range(len(frames)):
    imgs = [res[r.name].output_images[i] for r in representations]
    collage = collage_fn(imgs, titles=[r.name for r in representations], size_px=70, rows_cols=None)
    image_write(collage, f"collage_{frames[i]}.png")
    plt.figure(figsize=(20, 10))
    plt.imshow(collage)
    plt.show()
